<a href="https://colab.research.google.com/github/bwsi-hadr/2019-student-final-exercise/blob/master/Planning_Facility_Flooding_and_Set_Cover_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# need to specify location of some certificates for rasterio
!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
!sudo mkdir -p /etc/pki/tls/certs
!sudo cp /etc/ssl/certs/ca-certificates.crt /etc/pki/tls/certs/ca-bundle.crt
try:
  import rasterio
  import rasterio.plot
  import rasterio.merge 
  import rasterio.mask
except:
  !pip install rasterio
  import rasterio
  import rasterio.plot
  import rasterio.merge
  import rasterio.mask
  
try:
  import rasterstats as rs
except:
  !pip install rasterstats  
  import rasterstats as rs
  
try:
  import pyproj
except:
  !pip install pyproj
  import pyproj
  
import networkx as nx
try:
  import osmnx as ox
except:
  # osmnx depends on the system package libspatialindex
  !apt install libspatialindex-dev
  !pip install osmnx
  import osmnx as ox

try: 
  import geopandas as gpd
except: 
  !pip install geopandas 
  import geopandas as gpd
  
try:
  import contextily as ctx 
except:
  # install dependencies for contextily
  !apt install libproj-dev proj-data proj-bin
  !apt install libgeos-dev
  !pip install cython
  !pip install cartopy
  # install contextily
  !pip install contextily==1.0rc1 --no-use-pep517 --no-cache-dir
  import contextily as ctx
  
import fiona
from shapely.geometry import Point, LineString, Polygon
  
import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/BWSI-Remote-Sensing/'Final_exercise'

     |████████████████████████████████| 19.7MB 2.8MB/s 
     |████████████████████████████████| 17.9MB 3.7MB/s 
     |████████████████████████████████| 81kB 22.4MB/s 
  Created wheel for simplejson: filename=simplejson-3.16.0-cp36-cp36m-linux_x86_64.whl size=114016 sha256=48f2d82b965786ab0075070d109e4a8afd8fb548beb43095b7c0afbc76c0fffd
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6613 sha256=f2360b258cda7d891edd27223546c849a8817687c88d67ca52a129ae6f5d5643
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built simplejson munch
     |████████████████████████████████| 11.2MB 2.5MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt 

In [4]:
!ls game_grid_export

facilities	      flooding.prj	       game_grid_all_stats.dbf
facilities_stats.csv  flooding.shp	       game_grid_all_stats.prj
flooding.cpg	      flooding.shx	       game_grid_all_stats.shp
flooding.dbf	      game_grid_all_stats.cpg  game_grid_all_stats.shx


In [0]:
grid = gpd.read_file('game_grid_export/game_grid_all_stats.shp').to_crs(epsg = 3857)
grid['centroid'] = grid['geometry'].centroid

In [0]:
to_m = lambda km: km * 1000

#Hospitals

In [7]:
hospital_grid = gpd.read_file('game_grid_export/facilities/hospitals.shp').to_crs(grid.crs)
hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] >= 200, 'hos_radius (km)'] = 25
hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] < 200, 'hos_radius (km)'] = 15

big_hospitals=hospital_grid[hospital_grid['BEDS']>=200].to_crs(grid.crs)
small_hospitals=hospital_grid[hospital_grid['BEDS']<200].to_crs(grid.crs)
hospitals_buffered = hospital_grid.copy().to_crs(grid.crs)
hospitals_buffered.loc[big_hospitals.index, 'geometry'] = hospitals_buffered.loc[big_hospitals.index,'geometry'].buffer(to_m(25))
hospitals_buffered.loc[small_hospitals.index, 'geometry'] = hospitals_buffered.loc[small_hospitals.index,'geometry'].buffer(to_m(15))

hosp_info = gpd.sjoin(grid, hospitals_buffered, how = 'inner', op = 'intersects')
hosp_infopop = hosp_info.groupby('NAME').sum()

alpha_hosp = hospitals_buffered.sort_values('NAME').set_index('NAME')
alpha_hosp['population'] = hosp_infopop['population'].values
alpha_orhosp = hospital_grid.sort_values('NAME').set_index('NAME')
alpha_hosp['centroid'] = alpha_orhosp['geometry'].values

alpha_hosp['population']

NAME
ARBOUR FULLER HOSPITAL                                      216673
ARBOUR HOSPITAL, THE                                        778984
ARBOUR HUMAN RESOURCE INSTITUTE                             869992
BAY RIDGE HOSPITAL BEVERLY                                  352378
BETH ISRAEL DEACONESS HOSPITAL - NEEDHAM                    330194
BETH ISRAEL DEACONESS HOSPITAL MILTON                       501945
BETH ISRAEL DEACONESS HOSPITAL PLYMOUTH                      38508
BETH ISRAEL DEACONESS MEDICAL CENTER - EAST CAMPUS         1208502
BETH ISRAEL DEACONESS MEDICAL CENTER - WEST CAMPUS         1200722
BOSTON MEDICAL CENTER - EAST NEWTON CAMPUS                 1216527
BOSTON MEDICAL CENTER - MENINO CAMPUS                      1217093
BOURNEWOOD HOSPITAL                                         712321
BRIGHAM AND WOMEN'S HOSPITAL                               1204412
CAMBRIDGE HEALTH ALLIANCE - CAMBRIDGE CAMPUS               1222459
CAMBRIDGE HEALTH ALLIANCE - SOMERVILLE CAMPUS            

#EMS

In [8]:
EMS_grid = gpd.read_file('game_grid_export/facilities/EMS.shp').to_crs(grid.crs)
EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] >= 5, 'EMS_radius (km)'] = 15
EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] < 5, 'EMS_radius (km)'] = 10

big_EMS=EMS_grid[EMS_grid['TOTAL_VEHI']>=5].to_crs(grid.crs)
small_EMS=EMS_grid[EMS_grid['TOTAL_VEHI']<5].to_crs(grid.crs)
EMS_buffered = EMS_grid.copy().to_crs(grid.crs)
EMS_buffered.loc[big_EMS.index, 'geometry'] = EMS_buffered.loc[big_EMS.index,'geometry'].buffer(to_m(15))
EMS_buffered.loc[small_EMS.index, 'geometry'] = EMS_buffered.loc[small_EMS.index,'geometry'].buffer(to_m(10))

EMS_info = gpd.sjoin(grid, EMS_buffered, how = 'inner', op = 'intersects')
EMS_infopop = EMS_info.groupby('NAME').sum()

alpha_EMS = EMS_buffered.sort_values('NAME').set_index('NAME')
alpha_EMS = alpha_EMS.loc[~alpha_EMS.index.duplicated(keep='first')]
alpha_EMS.loc[EMS_infopop.index,'population'] = EMS_infopop['population'].values
alpha_EMS = alpha_EMS.dropna(subset=['population'])

alpha_EMS['population']

NAME
ABINGTON FIRE DEPARTMENT STATION 1                               100719.0
ABINGTON FIRE DEPARTMENT STATION 2                               183927.0
ACTION AMBULANCE SERVICE - HEADQUARTERS                          204271.0
ACTION AMBULANCE SERVICE - LYNN                                  177090.0
ACTION AMBULANCE SERVICE - STONEHAM                              431782.0
ACTION AMBULANCE SERVICE - SWAMPSCOTT                            192341.0
ACTION AMBULANCE SERVICE - WAKEFIELD                             371726.0
ACTION AMBULANCE SERVICE - WOBURN                                196171.0
ACTON FIRE DEPARTMENT STATION 1                                   67473.0
ACTON FIRE DEPARTMENT STATION 2                                   67614.0
ACTON FIRE DEPARTMENT STATION 3                                   36307.0
ACUSHNET EMERGENCY MEDICAL SERVICES                               82096.0
ACUSHNET FIRE AND RESCUE STATION 1 - HEADQUARTERS                111806.0
ACUSHNET FIRE AND RESCUE STATION 

#Shelters

In [9]:
shelt_grid = gpd.read_file('game_grid_export/facilities/shelters.shp').to_crs(grid.crs)
shelt_grid['shelt_radius (km)'] = 3
shelt_buffered = shelt_grid.copy().to_crs(grid.crs)
shelt_buffered['geometry'] = shelt_buffered['geometry'].buffer(to_m(3))

shelt_info = gpd.sjoin(grid, shelt_buffered, how = 'inner', op = 'intersects')
shelt_infopop = shelt_info.groupby('NAME').sum()

alpha_shelt = shelt_buffered.sort_values('NAME').set_index('NAME')
alpha_shelt = alpha_shelt.loc[~alpha_shelt.index.duplicated(keep='first')]
alpha_shelt.loc[shelt_infopop.index,'population'] = shelt_infopop['population'].values
alpha_shelt = alpha_shelt.dropna(subset=['population'])

alpha_shelt['population']

NAME
ABINGTON HIGH SCHOOL                                  9137.0
ABINGTON SENIOR CENTER                               14835.0
ACTON SENIOR CENTER                                   7603.0
ADAMS MIDDLE SCHOOL                                  28433.0
ADRIAN TINSLEY CENTER / BRIDGEWATER STATE COLLEGE    10563.0
AGASSIZ COMMUNITY CENTER                             58607.0
AHERN MIDDLE SCHOOL                                   7050.0
AIKEN ELEMENTARY SCHOOL                              10480.0
ALBERT F. FORD MIDDLE SCHOOL                          2726.0
ALDEN ELEMENTARY SCHOOL                               3103.0
ALFRED LIMA ELEMENTARY SCHOOL                        62965.0
AMERICAL CIVIC CENTER                                20029.0
ANGELO ELEMENTARY SCHOOL                             35089.0
ANNA M. MCCABE SCHOOL                                 7414.0
ANTHONY CARNEVALE JR. ELEMENTARY SCHOOL              48654.0
AQUINNAH TOWN HALL                                     202.0
ARLINGTON CATHOLIC 

#Fire Stations

In [10]:
fire_grid = gpd.read_file('game_grid_export/facilities/fire_stations.shp').to_crs(grid.crs)
fire_grid['fire_radius (km)'] = 15
fire_buffered = fire_grid.copy().to_crs(grid.crs)
fire_buffered['geometry'] = fire_buffered['geometry'].buffer(to_m(15))

fire_info = gpd.sjoin(grid, fire_buffered, how = 'inner', op = 'intersects')
fire_infopop = fire_info.groupby('NAME').sum()

alpha_fire = fire_buffered.sort_values('NAME').set_index('NAME')
alpha_fire = alpha_fire.loc[~alpha_fire.index.duplicated(keep='first')]
alpha_fire.loc[fire_infopop.index,'population'] = fire_infopop['population'].values
alpha_fire = alpha_fire.dropna(subset=['population'])

alpha_fire['population']

NAME
ABINGTON FIRE DEPARTMENT STATION 1                             210174.0
ABINGTON FIRE DEPARTMENT STATION 2                             183927.0
ACTON FIRE DEPARTMENT STATION 1                                 67473.0
ACTON FIRE DEPARTMENT STATION 2                                 67614.0
ACTON FIRE DEPARTMENT STATION 3                                 66696.0
ACUSHNET FIRE AND RESCUE STATION 1 - HEADQUARTERS              111806.0
ACUSHNET FIRE AND RESCUE STATION 2                              79389.0
ALBION FIRE DEPARTMENT                                         183429.0
ALDEN MILLS POINT OF PINES FIRE STATION - TRAINING FACILITY    374033.0
AQUINNAH FIRE DEPARTMENT                                          826.0
ARLINGTON FIRE DEPARTMENT - HEADQUARTERS                       733870.0
ARLINGTON FIRE DEPARTMENT - HIGHLAND STATION                   667193.0
ARLINGTON FIRE DEPARTMENT - PARK CIRCLE STATION                621716.0
ASHLAND FIRE DEPARTMENT STATION 1                          

#Set Cover Application For Any Single Facility


In [0]:
def calc_weight(subset, size, cost_ref):
  return sum(grid.loc[cell, 'population'] for cell in subset) / cost_ref[size]

In [0]:
#freq_chart = [0 for _ in range(game_grid.shape[0])]
covered = set()

def set_cover(budget, facdict, cost_ref): #facdict = hosp_to_cells, EMS_to_cells, shelt_to_cells, fire_to_cells
  to_ret = set()
  while budget > 0 and len(facdict.items()) != 0:
    max_fac, max_subset = max(facdict.items(), key = lambda tup: \
                               calc_weight(tup[1], tup[0][1], cost_ref))
    
#Set cover could be wrong because of overlapping areas
#    for cell in max_subset:
#      freq_chart[cell] += 1
#       if freq_chart[cell] == 2:
#         done.add(cell)
    
    budget -= cost_ref[max_fac[1]]
    to_ret.add(max_fac)
    
    for key in facdict:
      facdict[key] -= max_subset
    del facdict[max_fac]
    
  return to_ret

In [0]:
#costs is a list of costs in the order of large hospitals, small hospitals
#large EMS, small EMS, shelters, and fire stations
def optfac (costs, budget, fac_info = hosp_info, fac = 'hospital'):
  large_hos_cost = costs[0]
  small_hos_cost = costs[1]
  large_ems_cost = costs[2]
  small_ems_cost = costs[3]
  shelter_cost = costs[4]
  fstation_cost = costs[5]

  cost_ref = {'smallhosp': small_hos_cost, 'largehosp': large_hos_cost, \
              'smallEMS': small_ems_cost, 'largeEMS': large_ems_cost, \
              'shelter': shelter_cost, 'fstation': fstation_cost}
  fac_to_cells = {}
  if fac == 'hospital':
    for i, row in fac_info.iterrows():
      hosp_name = row["NAME"]
      f = 'smallhosp' if row['hos_radius (km)'] == 15 else 'largehosp'
      if (hosp_name, f) not in fac_to_cells:
        fac_to_cells[(hosp_name, f)] = {i}
      else:
        fac_to_cells[(hosp_name, f)].add(i)
  elif fac == 'EMS':
    for i, row in fac_info.iterrows():
      EMS_name = row["NAME"]
      f = 'smallEMS' if row['EMS_radius (km)'] == 10 else 'largeEMS'
      if (EMS_name, f) not in fac_to_cells:
        fac_to_cells[(EMS_name, f)] = {i}
      else:
        fac_to_cells[(EMS_name, f)].add(i)
  elif fac == 'shelter':
    for i, row in fac_info.iterrows():
      shelt_name = row["NAME"]
      f = 'shelter'
      if (shelt_name, f) not in fac_to_cells:
        fac_to_cells[(shelt_name, f)] = {i}
      else:
        fac_to_cells[(shelt_name, f)].add(i)
  elif fac == 'firestation':
    for i, row in fac_info.iterrows():
      fire_name = row["NAME"]
      f = 'fstation'
      if (fire_name, f) not in fac_to_cells:
        fac_to_cells[(fire_name, f)] = {i}
      else:
        fac_to_cells[(fire_name, f)].add(i)
  else:
    return "no"
  
  return set_cover(budget, fac_to_cells, cost_ref)
    

In [113]:
#Example
#Costs as a list of costs in the order of large hospitals, small hospitals
#large EMS, small EMS, shelters, and fire stations; budget; facility type
#(default is hospital)
#facility options: 'hospital', 'EMS', 'shelter', 'firestation'
optfac([100, 50, 50, 25, 10, 15], 2000) 

{('LAHEY CLINIC MEDICAL CENTER', 'largehosp'): {2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2255, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2332, 2333, 2334, 2335, 302, 303, 304, 305, 306, 307, 3

{('ARBOUR FULLER HOSPITAL', 'smallhosp'),
 ('ARBOUR HOSPITAL, THE', 'smallhosp'),
 ('ARBOUR HUMAN RESOURCE INSTITUTE', 'smallhosp'),
 ('BAY RIDGE HOSPITAL BEVERLY', 'smallhosp'),
 ('BETH ISRAEL DEACONESS HOSPITAL - NEEDHAM', 'smallhosp'),
 ('BETH ISRAEL DEACONESS HOSPITAL MILTON', 'smallhosp'),
 ('CAMBRIDGE HEALTH ALLIANCE - SOMERVILLE CAMPUS', 'smallhosp'),
 ('CAMBRIDGE HEALTH ALLIANCE - WHIDDEN MEMORIAL CAMPUS', 'smallhosp'),
 ('CARITAS CARNEY HOSPITAL', 'smallhosp'),
 ('DANA-FARBER CANCER INSTITUTE', 'smallhosp'),
 ('EMERSON HOSPITAL', 'smallhosp'),
 ('EMMA PENDLETON BRADLEY HOSPITAL', 'smallhosp'),
 ('FAULKNER HOSPITAL', 'smallhosp'),
 ('GOOD SAMARITAN MEDICAL CENTER', 'largehosp'),
 ('HASBRO CHILDRENS HOSPITAL', 'smallhosp'),
 ('HEALTHSOUTH BRAINTREE REHABILITATION HOSPITAL', 'smallhosp'),
 ('HEALTHSOUTH NEW ENGLAND REHABILITATION HOSPITAL', 'smallhosp'),
 ('KENT HOSPITAL', 'largehosp'),
 ('LANDMARK MEDICAL CENTER', 'largehosp'),
 ('MASSACHUSETTS HOSPITAL SCHOOL', 'smallhosp'),
 (

#Proximity-to-Flood Function For Any Single Facility

In [0]:
flooding = gpd.read_file('game_grid_export/flooding.shp')

In [0]:
#Outputs a list of facilities in the specified distance from the flood
def prox(mindist, maxdist, flood, fac):
  fac_grid = None
  if fac == 'hospital':
    fac_grid = hospital_grid
  elif fac == 'EMS':
    fac_grid = EMS_grid
  elif fac == 'shelter':
    fac_grid = shelt_grid
  elif fac == 'firestation':
    fac_grid = fire_grid
  else:
    return "no"
  
  l = []
  for i in range(len(fac_grid)):
    dist = fac_grid.loc[i, 'geometry'].distance(flood)
    if dist <= maxdist and dist >= mindist: #TODO: mindist <= dist <= maxdist check
      l.append(fac_grid.loc[i, 'NAME'])
  return l

In [0]:
from shapely.geometry import MultiPolygon
import shapely.ops as shap
#Creates a MultiPolygon using the flood shapely file and the buffer size in meters
def flcreate(fl, b, i): 
  flooding = fl.copy().to_crs(grid.crs)
  flooding['geometry'] = flooding['geometry'].buffer(b)
  if i == 0:
    flood = flooding['geometry'].unary_union
  else:
    l = [MultiPolygon([flooding.loc[i, 'geometry'], \
                       flooding.loc[i + 1, 'geometry']]).convex_hull \
         for i in range(len(flooding) - 1)]
    flood = shap.cascaded_union(MultiPolygon(l))
    
  return flood
    

In [0]:
#Outputs all information for a list of facility names
def prettyUnicorn(l, fac):
  alpha_fac = None
  if fac == 'hospital':
    alpha_fac = alpha_hosp
  elif fac == 'EMS':
    alpha_fac = alpha_EMS
  elif fac == 'shelter':
    alpha_fac = alpha_shelt
  elif fac == 'firestation':
    alpha_fac = alpha_fire
  else:
    return "no"
  return alpha_fac.loc[l]

In [0]:
#Everything you'll need
#minimum distance, maximum distance, flood shapely file, buffer size in meters,
#type of graph (0: unary_union, 1: cascaded_union), facility type ('hospital', 'EMS', 'shelter', 'firestation')
def magic(mindist, maxdist, floodshp, buffer, graph_type = 0, fac = 'hospital'):
  return prettyUnicorn(prox(mindist, maxdist, flcreate(flooding, buffer, graph_type), fac), fac)

In [92]:
#Example

magic(0, 1000, flooding, 500, 0, 'hospital')

,OBJECTID,ID,ADDRESS,CITY,STATE,ZIP,TYPE,STATUS,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,WEBSITE,OWNER,TTL_STAFF,BEDS,HELIPAD,geometry,hos_radius (km),population,centroid
NAME,,,,,,,,,,,,,,,,,,,,,,,,
PROVIDENCE VA MEDICAL CENTER,24.0,0001102908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.831937,-71.433575,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,73.0,NOT AVAILABLE,POLYGON ((-7936949.155586868 5135837.564535611...,15.0,338290,POINT Z (-7951949.155586868 5135837.564535611 0)
"NAVAL HEALTH CLINIC NEW ENGLAND, NEWPORT",27.0,0001602841,43 SMITH ROAD,NEWPORT,RI,02841,GENERAL ACUTE CARE,OPEN,NEWPORT,44005,USA,41.503931,-71.321796,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://nhcne.med.navy.mil/locations/npri/npri.asp,GOVERNMENT - FEDERAL,-999.0,100.0,NOT AVAILABLE,"POLYGON ((-7924506.027807591 5086957.87205448,...",15.0,44084,POINT Z (-7939506.027807591 5086957.87205448 0)
VA MEDICAL CENTER - PROVIDENCE,28.0,0013402908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.832139,-71.433150,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,100.0,N,POLYGON ((-7936901.833110882 5135867.763963762...,15.0,337792,POINT Z (-7951901.833110882 5135867.763963762 0)
MEMORIAL HOSPITAL OF RHODE ISLAND,30.0,0000302860,111 BREWSTER STREET,PAWTUCKET,RI,02860,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.869922,-71.376456,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.mhri.org/,NON-PROFIT,-999.0,294.0,NOT AVAILABLE,"POLYGON ((-7920590.786097784 5141514.2622464, ...",25.0,499704,POINT Z (-7945590.786097784 5141514.2622464 0)
WOMEN AND INFANTS HOSPITAL OF RHODE ISLAND,32.0,0000702905,101 DUDLEY STREET,PROVIDENCE,RI,02905,WOMEN,OPEN,PROVIDENCE,44007,USA,41.810877,-71.412201,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.womenandinfants.org/,NON-PROFIT,-999.0,247.0,Y,"POLYGON ((-7924569.8250032 5132691.749874027, ...",25.0,506912,POINT Z (-7949569.8250032 5132691.749874027 0)
RHODE ISLAND HOSPITAL,34.0,0001002903,593 EDDY STREET,PROVIDENCE,RI,02903,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.811881,-71.409129,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.rhodeislandhospital.org/rhode-islan...,NON-PROFIT,-999.0,719.0,Y,"POLYGON ((-7924227.90673675 5132841.662859567,...",25.0,505346,POINT Z (-7949227.90673675 5132841.662859567 0)
EMMA PENDLETON BRADLEY HOSPITAL,35.0,0001202915,1011 VETERANS MEMORIAL PKWY,EAST PROVIDENCE,RI,02915,PSYCHIATRIC,OPEN,PROVIDENCE,44007,USA,41.792448,-71.367270,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.bradleyhospital.org/service-directo...,NON-PROFIT,-999.0,51.0,NOT AVAILABLE,POLYGON ((-7929568.200346095 5129939.701677736...,15.0,335609,POINT Z (-7944568.200346095 5129939.701677736 0)
HASBRO CHILDRENS HOSPITAL,37.0,0001502903,593 EDDY STREET,PROVIDENCE,RI,02903,CHILDREN,OPEN,PROVIDENCE,44007,USA,41.811150,-71.408906,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.hasbrochildrenshospital.org/,NON-PROFIT,-999.0,100.0,Y,"POLYGON ((-7934203.10357041 5132732.457129485,...",15.0,343961,POINT Z (-7949203.10357041 5132732.457129485 0)
ROGER WILLIAMS MEDICAL CENTER,39.0,0002002908,825 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.834839,-71.435390,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.rwmc.org/,NON-PROFIT,-999.0,177.0,NOT AVAILABLE,"POLYGON ((-7937151.19906801 5136271.097543607,...",15.0,337433,POINT Z (-7952151.19906801 5136271.097543607 0)


#Optimized Facility Flood Identification

In [0]:
#Outputs all information for a list of facility names
def ganggang(l, fac):
  fac_info = None
  if fac == 'hospital':
    fac_info = hosp_info
  elif fac == 'EMS':
    fac_info = EMS_info
  elif fac == 'shelter':
    fac_info = shelt_info
  elif fac == 'firestation':
    fac_info = fire_info
  else:
    return "no"
  

  return fac_info.loc[fac_info['NAME'].isin(l)]
  


In [0]:
def abracadaba(mindist, maxdist, floodshp, buffer, graph_type = 0, fac = 'hospital'):
  return ganggang(prox(mindist, maxdist, flcreate(flooding, buffer, graph_type), fac), fac)

In [0]:
def pewpew(costs, budget, mindist, maxdist, buffer, fl = flooding, graph_type = 0, fac = 'hospital'):
  x = optfac(costs, budget, prettyUnicorn2(prox(mindist, maxdist, flcreate(fl, buffer, graph_type), fac), fac))
  l = [i[0] for i in x]
  return prettyUnicorn(l, fac)

In [135]:
#Example

pewpew([100, 50, 50, 25, 10, 15], 2000, 0, 1000, 500, flooding, 0, 'hospital')

,OBJECTID,ID,ADDRESS,CITY,STATE,ZIP,TYPE,STATUS,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,WEBSITE,OWNER,TTL_STAFF,BEDS,HELIPAD,geometry,hos_radius (km),population,centroid
NAME,,,,,,,,,,,,,,,,,,,,,,,,
CAPE COD & ISLANDS COMMUNITY MENTAL HEALTH CENTER,84.0,0010702559,830 COUNTY RD,BOURNE,MA,02559,PSYCHIATRIC,OPEN,BARNSTABLE,25001,USA,41.688400,-70.602385,622210,"HOSPITALS, PSYCHIATRIC (EXCEPT CONVALESCENT)",NOT AVAILABLE,GOVERNMENT - STATE,-999.0,100.0,N,POLYGON ((-7844421.558232985 5114416.887640206...,15.0,37387,POINT Z (-7859421.558232985 5114416.887640206 0)
MARTHA'S VINEYARD HOSPITAL,179.0,0000702557,ONE HOSPITAL ROAD,OAK BLUFFS,MA,02557,GENERAL ACUTE CARE,OPEN,DUKES,25007,USA,41.460800,-70.579800,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.marthasvineyardhospital.org,NON-PROFIT,-999.0,25.0,Y,POLYGON ((-7841907.396309995 5080549.000211069...,15.0,15059,POINT Z (-7856907.396309995 5080549.000211069 0)
DR JOHN C CORRIGAN MENTAL HEALTH CENTER,61.0,0011402720,49 HILLSIDE STREET,FALL RIVER,MA,02720,PSYCHIATRIC,OPEN,BRISTOL,25005,USA,41.707487,-71.145714,622210,"HOSPITALS, PSYCHIATRIC (EXCEPT CONVALESCENT)",NOT AVAILABLE,GOVERNMENT - STATE,-999.0,100.0,NOT AVAILABLE,POLYGON ((-7904904.706361935 5117262.615547662...,15.0,119180,POINT Z (-7919904.706361935 5117262.615547662 0)
SOUTHCOAST HOSPITALS GROUP - CHARLTON MEMORIAL HOSPITAL,183.0,0005702720,363 HIGHLAND AVENUE,FALL RIVER,MA,02720,GENERAL ACUTE CARE,OPEN,BRISTOL,25005,USA,41.709400,-71.146000,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.southcoast.org/charlton/,NON-PROFIT,-999.0,743.0,Y,POLYGON ((-7894936.492044975 5117547.883270329...,25.0,235822,POINT Z (-7919936.492044975 5117547.883270329 0)
PROVIDENCE VA MEDICAL CENTER,24.0,0001102908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.831937,-71.433575,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,73.0,NOT AVAILABLE,POLYGON ((-7936949.155586868 5135837.564535611...,15.0,338290,POINT Z (-7951949.155586868 5135837.564535611 0)
CAPE COD HOSPITAL,189.0,0005502601,27 PARK STREET,BARNSTABLE,MA,02601,GENERAL ACUTE CARE,OPEN,BARNSTABLE,25001,USA,41.653400,-70.272900,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.capecodhealth.org/hospitals-facilit...,NON-PROFIT,-999.0,259.0,NOT AVAILABLE,POLYGON ((-7797743.444570526 5109201.015920504...,25.0,75378,POINT Z (-7822743.444570526 5109201.015920504 0)
WOMEN AND INFANTS HOSPITAL OF RHODE ISLAND,32.0,0000702905,101 DUDLEY STREET,PROVIDENCE,RI,02905,WOMEN,OPEN,PROVIDENCE,44007,USA,41.810877,-71.412201,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.womenandinfants.org/,NON-PROFIT,-999.0,247.0,Y,"POLYGON ((-7924569.8250032 5132691.749874027, ...",25.0,506912,POINT Z (-7949569.8250032 5132691.749874027 0)
SOUTHCOAST HOSPITALS GROUP - ST. LUKE'S HOSPITAL,181.0,0003602740,101 PAGE STREET,NEW BEDFORD,MA,02740,GENERAL ACUTE CARE,OPEN,BRISTOL,25005,USA,41.626400,-70.938400,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.southcoast.org/stlukes/,NON-PROFIT,-999.0,391.0,Y,"POLYGON ((-7871826.5657296 5105179.226333707, ...",25.0,152247,POINT Z (-7896826.5657296 5105179.226333707 0)
HASBRO CHILDRENS HOSPITAL,37.0,0001502903,593 EDDY STREET,PROVIDENCE,RI,02903,CHILDREN,OPEN,PROVIDENCE,44007,USA,41.811150,-71.408906,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.hasbrochildrenshospital.org/,NON-PROFIT,-999.0,100.0,Y,"POLYGON ((-7934203.10357041 5132732.457129485,...",15.0,343961,POINT Z (-7949203.10357041 5132732.457129485 0)


#Functions


##If you have no budget use magic, if you do have a budget use pewpew

In [136]:
#Example
#magic(mindist, maxdist, floodshp, buffer, graph_type = 0, fac = 'hospital')
magic(0, 1000, flooding, 500, 0, 'hospital')

,OBJECTID,ID,ADDRESS,CITY,STATE,ZIP,TYPE,STATUS,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,WEBSITE,OWNER,TTL_STAFF,BEDS,HELIPAD,geometry,hos_radius (km),population,centroid
NAME,,,,,,,,,,,,,,,,,,,,,,,,
PROVIDENCE VA MEDICAL CENTER,24.0,0001102908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.831937,-71.433575,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,73.0,NOT AVAILABLE,POLYGON ((-7936949.155586868 5135837.564535611...,15.0,338290,POINT Z (-7951949.155586868 5135837.564535611 0)
"NAVAL HEALTH CLINIC NEW ENGLAND, NEWPORT",27.0,0001602841,43 SMITH ROAD,NEWPORT,RI,02841,GENERAL ACUTE CARE,OPEN,NEWPORT,44005,USA,41.503931,-71.321796,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://nhcne.med.navy.mil/locations/npri/npri.asp,GOVERNMENT - FEDERAL,-999.0,100.0,NOT AVAILABLE,"POLYGON ((-7924506.027807591 5086957.87205448,...",15.0,44084,POINT Z (-7939506.027807591 5086957.87205448 0)
VA MEDICAL CENTER - PROVIDENCE,28.0,0013402908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.832139,-71.433150,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,100.0,N,POLYGON ((-7936901.833110882 5135867.763963762...,15.0,337792,POINT Z (-7951901.833110882 5135867.763963762 0)
MEMORIAL HOSPITAL OF RHODE ISLAND,30.0,0000302860,111 BREWSTER STREET,PAWTUCKET,RI,02860,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.869922,-71.376456,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.mhri.org/,NON-PROFIT,-999.0,294.0,NOT AVAILABLE,"POLYGON ((-7920590.786097784 5141514.2622464, ...",25.0,499704,POINT Z (-7945590.786097784 5141514.2622464 0)
WOMEN AND INFANTS HOSPITAL OF RHODE ISLAND,32.0,0000702905,101 DUDLEY STREET,PROVIDENCE,RI,02905,WOMEN,OPEN,PROVIDENCE,44007,USA,41.810877,-71.412201,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.womenandinfants.org/,NON-PROFIT,-999.0,247.0,Y,"POLYGON ((-7924569.8250032 5132691.749874027, ...",25.0,506912,POINT Z (-7949569.8250032 5132691.749874027 0)
RHODE ISLAND HOSPITAL,34.0,0001002903,593 EDDY STREET,PROVIDENCE,RI,02903,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.811881,-71.409129,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.rhodeislandhospital.org/rhode-islan...,NON-PROFIT,-999.0,719.0,Y,"POLYGON ((-7924227.90673675 5132841.662859567,...",25.0,505346,POINT Z (-7949227.90673675 5132841.662859567 0)
EMMA PENDLETON BRADLEY HOSPITAL,35.0,0001202915,1011 VETERANS MEMORIAL PKWY,EAST PROVIDENCE,RI,02915,PSYCHIATRIC,OPEN,PROVIDENCE,44007,USA,41.792448,-71.367270,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.bradleyhospital.org/service-directo...,NON-PROFIT,-999.0,51.0,NOT AVAILABLE,POLYGON ((-7929568.200346095 5129939.701677736...,15.0,335609,POINT Z (-7944568.200346095 5129939.701677736 0)
HASBRO CHILDRENS HOSPITAL,37.0,0001502903,593 EDDY STREET,PROVIDENCE,RI,02903,CHILDREN,OPEN,PROVIDENCE,44007,USA,41.811150,-71.408906,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.hasbrochildrenshospital.org/,NON-PROFIT,-999.0,100.0,Y,"POLYGON ((-7934203.10357041 5132732.457129485,...",15.0,343961,POINT Z (-7949203.10357041 5132732.457129485 0)
ROGER WILLIAMS MEDICAL CENTER,39.0,0002002908,825 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.834839,-71.435390,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.rwmc.org/,NON-PROFIT,-999.0,177.0,NOT AVAILABLE,"POLYGON ((-7937151.19906801 5136271.097543607,...",15.0,337433,POINT Z (-7952151.19906801 5136271.097543607 0)


In [137]:
#Example
#pewpew(costs, budget, mindist, maxdist, buffer, fl = flooding, graph_type = 0, fac = 'hospital')
pewpew([100, 50, 50, 25, 10, 15], 2000, 0, 1000, 500, flooding, 0, 'hospital')

,OBJECTID,ID,ADDRESS,CITY,STATE,ZIP,TYPE,STATUS,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,WEBSITE,OWNER,TTL_STAFF,BEDS,HELIPAD,geometry,hos_radius (km),population,centroid
NAME,,,,,,,,,,,,,,,,,,,,,,,,
CAPE COD & ISLANDS COMMUNITY MENTAL HEALTH CENTER,84.0,0010702559,830 COUNTY RD,BOURNE,MA,02559,PSYCHIATRIC,OPEN,BARNSTABLE,25001,USA,41.688400,-70.602385,622210,"HOSPITALS, PSYCHIATRIC (EXCEPT CONVALESCENT)",NOT AVAILABLE,GOVERNMENT - STATE,-999.0,100.0,N,POLYGON ((-7844421.558232985 5114416.887640206...,15.0,37387,POINT Z (-7859421.558232985 5114416.887640206 0)
MARTHA'S VINEYARD HOSPITAL,179.0,0000702557,ONE HOSPITAL ROAD,OAK BLUFFS,MA,02557,GENERAL ACUTE CARE,OPEN,DUKES,25007,USA,41.460800,-70.579800,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.marthasvineyardhospital.org,NON-PROFIT,-999.0,25.0,Y,POLYGON ((-7841907.396309995 5080549.000211069...,15.0,15059,POINT Z (-7856907.396309995 5080549.000211069 0)
DR JOHN C CORRIGAN MENTAL HEALTH CENTER,61.0,0011402720,49 HILLSIDE STREET,FALL RIVER,MA,02720,PSYCHIATRIC,OPEN,BRISTOL,25005,USA,41.707487,-71.145714,622210,"HOSPITALS, PSYCHIATRIC (EXCEPT CONVALESCENT)",NOT AVAILABLE,GOVERNMENT - STATE,-999.0,100.0,NOT AVAILABLE,POLYGON ((-7904904.706361935 5117262.615547662...,15.0,119180,POINT Z (-7919904.706361935 5117262.615547662 0)
SOUTHCOAST HOSPITALS GROUP - CHARLTON MEMORIAL HOSPITAL,183.0,0005702720,363 HIGHLAND AVENUE,FALL RIVER,MA,02720,GENERAL ACUTE CARE,OPEN,BRISTOL,25005,USA,41.709400,-71.146000,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.southcoast.org/charlton/,NON-PROFIT,-999.0,743.0,Y,POLYGON ((-7894936.492044975 5117547.883270329...,25.0,235822,POINT Z (-7919936.492044975 5117547.883270329 0)
PROVIDENCE VA MEDICAL CENTER,24.0,0001102908,830 CHALKSTONE AVE,PROVIDENCE,RI,02908,GENERAL ACUTE CARE,OPEN,PROVIDENCE,44007,USA,41.831937,-71.433575,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.providence.va.gov/,GOVERNMENT - FEDERAL,-999.0,73.0,NOT AVAILABLE,POLYGON ((-7936949.155586868 5135837.564535611...,15.0,338290,POINT Z (-7951949.155586868 5135837.564535611 0)
CAPE COD HOSPITAL,189.0,0005502601,27 PARK STREET,BARNSTABLE,MA,02601,GENERAL ACUTE CARE,OPEN,BARNSTABLE,25001,USA,41.653400,-70.272900,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.capecodhealth.org/hospitals-facilit...,NON-PROFIT,-999.0,259.0,NOT AVAILABLE,POLYGON ((-7797743.444570526 5109201.015920504...,25.0,75378,POINT Z (-7822743.444570526 5109201.015920504 0)
WOMEN AND INFANTS HOSPITAL OF RHODE ISLAND,32.0,0000702905,101 DUDLEY STREET,PROVIDENCE,RI,02905,WOMEN,OPEN,PROVIDENCE,44007,USA,41.810877,-71.412201,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.womenandinfants.org/,NON-PROFIT,-999.0,247.0,Y,"POLYGON ((-7924569.8250032 5132691.749874027, ...",25.0,506912,POINT Z (-7949569.8250032 5132691.749874027 0)
SOUTHCOAST HOSPITALS GROUP - ST. LUKE'S HOSPITAL,181.0,0003602740,101 PAGE STREET,NEW BEDFORD,MA,02740,GENERAL ACUTE CARE,OPEN,BRISTOL,25005,USA,41.626400,-70.938400,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.southcoast.org/stlukes/,NON-PROFIT,-999.0,391.0,Y,"POLYGON ((-7871826.5657296 5105179.226333707, ...",25.0,152247,POINT Z (-7896826.5657296 5105179.226333707 0)
HASBRO CHILDRENS HOSPITAL,37.0,0001502903,593 EDDY STREET,PROVIDENCE,RI,02903,CHILDREN,OPEN,PROVIDENCE,44007,USA,41.811150,-71.408906,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,http://www.hasbrochildrenshospital.org/,NON-PROFIT,-999.0,100.0,Y,"POLYGON ((-7934203.10357041 5132732.457129485,...",15.0,343961,POINT Z (-7949203.10357041 5132732.457129485 0)
